In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import string 

In [ ]:
COLUMN_NAMES = ["player_id", "name", "position", "team", "draft_year", "draft_round", "draft_pick_overall", "age_on_draft_day", "conference", "conference_strength", "height_in", "weight_lb", "forty_yd\nC_Games", "C_Rec", "C_RecYds", "C_RecTD", "C_RushAtt", "C_RushYds", "C_RushTD", "C_PassAtt", "C_PassYds", "C_PassTD", "C_INT\nCar_Games", "Car_Rec", "Car_RecYds", "Car_RecTD", "Car_RushAtt", "Car_RushYds", "Car_RushTD", "Car_PassAtt", "Car_PassYds", "Car_PassTD", "Car_INT\nTeam_RecYds", "Team_RecTD", "Team_RushYds", "Team_RushTD", "Team_PassYds", "Team_PassTD\nRec", "RecYds", "RecTD", "RushYds", "RushTD", "PassYds", "PassTD", "Fumbles", "INT"]
df = pd.DataFrame(columns=COLUMN_NAMES)
print(df)

Empty DataFrame
Columns: [player_id, name, position, team, draft_year, draft_round, draft_pick_overall, age_on_draft_day, conference, conference_strength, height_in, weight_lb, forty_yd
C_Games, C_Rec, C_RecYds, C_RecTD, C_RushAtt, C_RushYds, C_RushTD, C_PassAtt, C_PassYds, C_PassTD, C_INT
Car_Games, Car_Rec, Car_RecYds, Car_RecTD, Car_RushAtt, Car_RushYds, Car_RushTD, Car_PassAtt, Car_PassYds, Car_PassTD, Car_INT
Team_RecYds, Team_RecTD, Team_RushYds, Team_RushTD, Team_PassYds, Team_PassTD
Rec, RecYds, RecTD, RushYds, RushTD, PassYds, PassTD, Fumbles, INT]
Index: []

[0 rows x 46 columns]


Download Excel files for the drafts of the last 5 years from https://www.pro-football-reference.com/

Rnd -- Round selected in draft
Supplemental draft round indicated with 'S'
Pick -- Overall Selection in Draft
Pos -- Position
Age -- Age as of September 1 of the draft year
To -- Last Year
Misc
AP1 -- First-team all-pro selections
PB -- Pro Bowl Selections
St -- Number of years as primary starter for his team at his position
Approx Val
wAV -- Weighted Career Approximate Value.
See the glossary for details on how we compute Approximate Value.
The career AV is computed by summing
100 percent of the AV of his best season,
95 percent of the AV of his next-best season,
90 percent of the AV of his third-best season,
and so on
DrAV -- Weighted AV accumulated for team that drafted this player
Games
G -- Games played
Passing
Cmp -- Passes completed
Att -- Passes attempted
Yds -- Yards Gained by Passing
For teams, sack yardage is deducted from this total
TD -- Passing Touchdowns
Int -- Interceptions thrown
Rushing
Att -- Rushing Attempts (sacks not included in NFL)
Yds -- Rushing Yards Gained (sack yardage is not included by NFL)
TD -- Rushing Touchdowns
Receiving
Rec -- Receptions
Yds -- Receiving Yards
TD -- Receiving Touchdowns
Tackles
Solo -- Tackles
Before 1994: unofficial and inconsistently recorded from team to team. For amusement only.
1994-now: unofficial but consistently recorded.
Def Interceptions
Int -- Passes intercepted on defense
Sk -- Sacks (official since 1982,
based on play-by-play, game film
and other research since 1960)

In [14]:
# Get Drafted College Football Players from the last 5 years 
import re

df_2024 = pd.read_csv("2024_draft.csv")
df_2023 = pd.read_csv("2023_draft.csv")

df_years = {"2023": df_2023, "2024": df_2024}
player_names = []

for year in ["2023", "2024"]:
    for name in df_years[year]["Player"]:
        if isinstance(name, str):  # Only process if it's a string
            clean_name = re.sub(r'[^\w\s]', '', name.lower())  # remove punctuation
            player_names.append(clean_name.replace(" ", "-"))

print(player_names[:5])



['bryce-young', 'cj-stroud', 'will-anderson', 'anthony-richardson', 'devon-witherspoon']


In [ ]:

def get_draft_info()
    years = ["2019", "2020", "2021", "2022", "2023", "2024"]
    url = ""
    player_names = []
    for year in years:
        url = "https://www.pro-football-reference.com/years/" + year + "/draft.htm"
    

In [21]:
def get_player_pages():
    alphabet = string.ascii_lowercase
    url = ""
    player_pages = []
    for letter in alphabet:
        url = "https://www.sports-reference.com/cfb/players/" + letter + "-index.html"

        response = requests.get(url)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}")
            return []
        response = response.text
        soup = BeautifulSoup(response, "html.parser")

        container = soup.find("div", {"id": "div_players"})
    
        for p in container.find_all("p"):
            a = p.find("a") # first link is for player page (second for school)
            if a and "/cfb/players/" in a["href"]:
                player_name = a.text.strip()
                player_url = "https://www.sports-reference.com" + a["href"]
                player_pages.append({"name": player_name, "url": player_url})
    return player_pages

# base = "https://www.sports-reference.com"
# index_url = f"{base}/cfb/players/a-index.html"

# html = requests.get(index_url).text
# soup = BeautifulSoup(html, "html.parser")

# player_links = []
# for p in soup.select("div#div_players p"):
#     a = p.find("a")
#     if a and "/cfb/players/" in a["href"]:
#         player_links.append(base + a["href"])


player_pages = get_player_pages()
print(player_pages[:5])
# print(len(player_pages))



Request failed with status 429
[]


In [ ]:
def get_player_stats():
    for name in player_names:
        found = False 
        for page in range(1,20): 
            url = f"https://www.sports-reference.com/cfb/players/{name}-{page}.html"
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    tables = pd.read_html(url)
                    print(f"Found data for {name} at {url}")
                    # Do something with tables here!!!!
                    found = True
                    break
            except Exception as e:
                print(f"Error fetching {url}: {e}")
        if not found:
            print(f"No page found for {name}")

    return tables 


base = "https://www.sports-reference.com"
index_url = f"{base}/cfb/players/a-index.html"

html = requests.get(index_url).text
soup = BeautifulSoup(html, "html.parser")

player_links = []
for p in soup.select("div#div_players p"):
    a = p.find("a")
    if a and "/cfb/players/" in a["href"]:
        player_links.append(base + a["href"])

        
tables = get_player_stats()
print(len(tables))
# print(tables[0].head())
# print(tables)

No page found for bryce-young


KeyboardInterrupt: 